In [ ]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd

from gene_benchmark.descriptor import NCBIDescriptor,BasePairDescriptor
from gene_benchmark.encoder import SentenceTransformerEncoder
from gene_benchmark.tasks import EntitiesTask

# Create embeddings & descriptions
This package enables turning biological text data and encoding it numerically using language models. The package also includes means to retrieve biological data.
See the README about further details about creation of descriptions and encoding

In [ ]:
prompts_maker = NCBIDescriptor()
prompts = prompts_maker.describe(entities=pd.Series(["BRCA1", "FOSL2"]))
print(prompts.loc[0])

We can look at the object summary

In [ ]:
print(prompts_maker.summary())

In [ ]:
bp = BasePairDescriptor()
bp.describe(pd.Series("TMEM92"))


Now we can take the text and encode it

In [ ]:
prompts = pd.Series(["ACTG"])

In [ ]:
import os


mpnet_name = "sentence-transformers/all-mpnet-base-v2"
mpnet_name =  "dsfsi/simcse-dna"
#mpnet_name =  "tydymy/DNABERT-2-117M-finetuned-human-150bp-dna-classification"
single_symbol_encoding = SentenceTransformerEncoder(mpnet_name)
encoding = single_symbol_encoding.encode(prompts)
print(encoding)

We can look at the encoder summary

In [ ]:
print(single_symbol_encoding.summary())

The package comes with a set of sum pre-defined tasks with entities and outcomes and a pipeline object to go from entities (currently just one gene symbol) to prompt to embeddings. Following is a simple example:

In [ ]:
biolink_name = "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
task = EntitiesTask(
    "long vs short range TF", encoder=biolink_name, description_builder=NCBIDescriptor()
)
res = task.run()
print(res)

# Existing tasks
See the readme for further details about the tasks.

In [ ]:
task = EntitiesTask(
    task="long vs short range TF",
    description_builder=prompts_maker,
    encoder=SentenceTransformerEncoder(mpnet_name),
)
_ = task.run()
task.summary()